Screen data to tabular form

In [1]:
text = open('list.txt', 'r', encoding = 'utf-8')
to_analyse = text.read()

In [84]:
to_analyse_list = to_analyse.split('\n')
display(to_analyse_list)

['1. ภาษาไทย ',
 'มีจำนวน 4 สาระ 4 มาตรฐาน 27 ตัวชี้วัด',
 'มาตรฐาน ท 1.2 การเขียน',
 '\tใช้กระบวนการเขียนเขียนสื่อสาร เขียนเรียงความ ย่อความ และเขียนเรื่องราวในรูปแบบต่าง ๆ เขียนรายงานข้อมูลสารสนเทศและรายงานการศึกษาค้นคว้าอย่างมีประสิทธิภาพ',
 '\tท 2.1 ม.4-6/1 เขียนสื่อสารในรูปแบบต่าง ๆ ได้ตรงตามวัตถุประสงค์ โดยใช้ภาษา เรียบเรียงถูกต้อง มีข้อมูล และสาระสำคัญชัดเจน',
 '\tท 2.1 ม.4-6/2 เขียนเรียงความ',
 '\tท 2.1 ม.4-6/3 เขียนย่อความจากสื่อที่มีรูปแบบและเนื้อหาหลากหลาย',
 '\tท 2.1 ม.4-6/4 ผลิตงานเขียนของตนเองในรูปแบบต่าง ๆ',
 '\tท 2.1 ม.4-6/5 ประเมินงานเขียนของผู้อื่น แล้วนำมาพัฒนางานเขียนของตนเอง',
 '\tท 2.1 ม.4-6/6 เขียนรายงานการศึกษาค้นคว้าเรื่องที่สนใจ ตามหลักการเขียนเชิงวิชาการและใช้ข้อมูลสารสนเทศอ้างอิงอย่างถูกต้อง',
 '\tท 2.1 ม.4-6/7 บันทึกการศึกษาค้นคว้า เพื่อนำไปพัฒนาตนเองอย่างสม่ำเสมอ',
 '\tท 2.1 ม.4-6/8 มีมารยาทในการเขียน',
 'มาตรฐาน ท 1.3 การฟัง การดู และการพูด',
 '\tสามารถเลือกฟังและดูอย่างมีวิจารณญาณ และพูดแสดงความรู้ ความคิด และความรู้สึกในโอกาสต่าง ๆ อย่างมีวิจารณญาณและสร

In [111]:
import pandas as pd

objectives = []
subsection = []
standards = []
subjects = []
include = ['คณิตศาสตร์', 'วิทยาศาสตร์', 'สังคมศึกษา', 'สุขศึกษาและพลศึกษา', 'ศิลปะ', 'การงานอาชีพ', 'ภาษาต่างประเทศ']

for i in to_analyse_list:
    tmp = ' '
    if i == '': continue
    elif i[0].isnumeric():
        subjects.append(i.split()[1])
    elif subjects[-1] in include and i[:4] == 'สาระ':
        subsection.append({'วิชา': subjects[-1], 'สาระ': tmp.join(i.split()[2:])})
    elif i[:7] == 'มาตรฐาน' and subjects[-1] in include:
        standards.append({'วิชา': subjects[-1], 'สาระ': subsection[-1]['สาระ'], 'มาตรฐาน': tmp.join(i.split()[3:])})
    elif i[:7] == 'มาตรฐาน':
        standards.append({'วิชา': subjects[-1], 'มาตรฐาน': tmp.join(i.split()[3:])})
    elif i[:1] == '\t':
        if i[2] == ' ' or i[1].isnumeric():
            tmp2 = i[1:].split()
            objectives.append({'มาตรฐาน' : standards[-1]['มาตรฐาน'], 'ตัวชี้วัด' : tmp.join(tmp2[1:]) if tmp2[0][0].isnumeric() else tmp.join(tmp2[3:])})

In [5]:
import pandas as pd

object_pd = pd.read_csv('Std51\objectives.csv', encoding='UTF-8')
standard_pd = pd.read_csv('Std51\standards.csv', encoding='UTF-8')
subject_pd = pd.read_csv('Std51\subjects.csv', encoding='UTF-8')
subsection_pd = pd.read_csv('Std51\subsection.csv', encoding='UTF-8')

In [6]:
import pythainlp
from pythainlp import tokenize, spell

obj_word = []
for i in object_pd['ตัวชี้วัด']:
    obj_word.append(tokenize.word_tokenize(i, engine='deepcut', keep_whitespace = False))

In [7]:
import ast

def ordered_sublist(list1, list2):
    if list1[0] not in list2: return False, -1
    else:
        idx_list = []
        count = list2.count(list1[0])
        for idx in range(count):
            firstidx = list2.index(list1[0], idx)
            for i in range(len(list1)):
                if list1[i] != list2[firstidx + i]: break
                idx_list.append(firstidx)
                break
    if idx_list == []: return False, -1
    return True, idx_list

def join_list(joindict, testlist):
    for i,j in joindict.items():
        tmp = ast.literal_eval(i)
        condition = ordered_sublist(tmp, testlist)
        if condition[1] != -1: condition[1].reverse()
        if condition[0]:
            for idx in condition[1]:
                testlist[idx:idx+len(tmp)] = j
    return testlist

In [35]:
dict_freq = {}
for i in range(len(obj_word)):
    tmp = set(obj_word[i])
    for j in list(tmp):
        try:
            dict_freq[j].append(i)
        except:
            dict_freq[j] = [i]
dict_count = {}
for i, j in dict_freq.items():
    dict_count[i] = len(j)
sort_value = pd.Series(dict_count).to_frame().reset_index().sort_values(0, ascending = False)

,index,0
13,และ,454
32,ของ,338
50,การ,330
24,ที่,251
86,อธิบาย,237
...,...,...
793,นิทรรศการ,1
794,15,1
795,ประชุม,1
796,ชิ้น,1


In [9]:
from pythainlp.tag import pos_tag
removal = sort_value[sort_value.apply(lambda x: pos_tag([x['index']])[0][1] not in ['NCMN', 'NPRP', 'VACT'], axis = 1)]['index'].tolist()
for i in removal:
    del dict_freq[i]

In [40]:
newdf = pd.merge(subject_pd, pd.merge(subsection_pd, pd.merge(standard_pd, object_pd), how='right'), how='left')
universal = {}
for i, j in dict_freq.items():
    for k in j:
        value = newdf.iloc[k]['สาระ'] if newdf.iloc[k]['วิชา'] == 'วิทยาศาสตร์' else newdf.iloc[k]['วิชา']
        try: universal[i].add(value)
        except: universal[i] = set(); universal[i].add(value)
universal

ก ดาราศาสตร์ และอวกาศ'},
 'ประชากร': {'ชีววิทยา'},
 'ถี่': {'ชีววิทยา', 'ฟิสิกส์'},
 'ฮาร์ดี': {'ชีววิทยา'},
 'สมดุล': {'ชีววิทยา', 'ฟิสิกส์', 'เคมี', 'โลก ดาราศาสตร์ และอวกาศ'},
 'สปีชีส์': {'ชีววิทยา'},
 'นิเวศ': {'ชีววิทยา'},
 'ชีวภาพ': {'ชีววิทยา'},
 'เริ่ม': {'ชีววิทยา', 'เคมี'},
 'กลุ่มโพรทิสต์': {'ชีววิทยา'},
 'กลุ่มแบคทีเรีย': {'ชีววิทยา'},
 'ไจ': {'ชีววิทยา'},
 'กลุ่มพืช': {'ชีววิทยา'},
 'กลุ่มฟัง': {'ชีววิทยา'},
 'วิทยาศาสตร์': {'ชีววิทยา', 'สังคมศึกษา', 'เคมี', 'เทคโนโลยี'},
 'จำแนก': {'ชีววิทยา', 'ศิลปะ', 'โลก ดาราศาสตร์ และอวกาศ'},
 'จนถึง': {'ชีววิทยา'},
 'ชื่อ': {'ชีววิทยา', 'เคมี', 'โลก ดาราศาสตร์ และอวกาศ'},
 'ย่อย': {'ชีววิทยา', 'เคมี'},
 'หมวด': {'ชีววิทยา'},
 'ขั้น': {'ชีววิทยา'},
 'ไดโคโทมัสคีย์': {'ชีววิทยา'},
 'เนื้อเยื่อ': {'ชีววิทยา'},
 'แผนผัง': {'ชีววิทยา'},
 'คู่': {'ชีววิทยา', 'ฟิสิกส์', 'ศิลปะ', 'สุขศึกษาและพลศึกษา', 'เคมี'},
 'ขวาง': {'ชีววิทยา'},
 'เลี้ยง': {'ชีววิทยา'},
 'เดี่ยว': {'ชีววิทยา', 'ฟิสิกส์', 'ศิลปะ', 'เคมี'},
 'ตัด': {'ชีววิทยา', 'ฟิสิกส์'}

In [43]:
filtration = {}
for i, j in universal.items():
    cond_set = {subject_pd['วิชา'].iloc[0], subject_pd['วิชา'].iloc[7]}
    if j == cond_set:
        filtration[i] = len(dict_freq[i])
print(cond_set)

{'ภาษาไทย', 'ภาษาต่างประเทศ'}


In [48]:
import json

to_json = {'elements': [], 'connection': []}
subjects = {'ภาษาไทย':'Thai Linguistics', 'คณิตศาสตร์':'Mathematics', 'ชีววิทยา':'Biology', 'เคมี':'Chemistry', 'ฟิสิกส์':'Physics',
            'โลก ดาราศาสตร์ และอวกาศ':'Geology, Astronomy, and Cosmology', 'เทคโนโลยี':'Technology', 'สังคมศึกษา':'Social Science',
            'สุขศึกษาและพลศึกษา':'Health Science & Physical Education', 'ศิลปะ':'Arts', 'การงานอาชีพ':'Home Economics',
           'ภาษาต่างประเทศ':'Foreign Linguistics'}

for i, j in universal.items():
    tmp_elements = {}
    tmp_elements['label'] = i
    tmp_elements['count'] = dict_count[i]
    tmp_elements['tags'] = []
    for k in j: tmp_elements['tags'].append(subjects[k])
    to_json['elements'].append(tmp_elements)

with open('course.json', 'w', encoding = 'utf8') as json_file:
    json.dump(to_json, json_file, ensure_ascii = False)

In [28]:
to_json

om': 'ตัวแปร', 'to': 'คณิตศาสตร์'},
  {'from': 'เหตุการณ์', 'to': 'วิทยาศาสตร์'},
  {'from': 'เหตุการณ์', 'to': 'ภาษาต่างประเทศ'},
  {'from': 'เหตุการณ์', 'to': 'คณิตศาสตร์'},
  {'from': 'เหตุการณ์', 'to': 'สังคมศึกษา'},
  {'from': 'แจก', 'to': 'คณิตศาสตร์'},
  {'from': 'เอก', 'to': 'คณิตศาสตร์'},
  {'from': 'ตรวจสอบ', 'to': 'วิทยาศาสตร์'},
  {'from': 'ตรวจสอบ', 'to': 'คณิตศาสตร์'},
  {'from': 'ตรวจสอบ', 'to': 'สังคมศึกษา'},
  {'from': 'พีชคณิต', 'to': 'คณิตศาสตร์'},
  {'from': 'หาอนุพันธ์', 'to': 'คณิตศาสตร์'},
  {'from': 'เขต', 'to': 'วิทยาศาสตร์'},
  {'from': 'เขต', 'to': 'คณิตศาสตร์'},
  {'from': 'จำกัด', 'to': 'คณิตศาสตร์'},
  {'from': 'หาปริพันธ์', 'to': 'คณิตศาสตร์'},
  {'from': 'ทำ', 'to': 'วิทยาศาสตร์'},
  {'from': 'ทำ', 'to': 'สุขศึกษาและพลศึกษา'},
  {'from': 'ทำ', 'to': 'การงานอาชีพ'},
  {'from': 'ทำ', 'to': 'สังคมศึกษา'},
  {'from': 'สัมพันธ์', 'to': 'วิทยาศาสตร์'},
  {'from': 'สัมพันธ์', 'to': 'ภาษาต่างประเทศ'},
  {'from': 'สัมพันธ์', 'to': 'สังคมศึกษา'},
  {'from': 'ดำรง'